In [1]:
from meta import *
from utils import *

In [7]:
idx = get_file_audio_ids(params_path[BASS][NO_FX])[0]
# idx
wets = [x for x in os.listdir(params_path[BASS][OVERDRIVE]) if x.startswith(idx)]


In [10]:
wets[0].split(params_data['separator'])[2].endswith('2')

True

In [2]:
get_input_target_pairs(GUITAR, DISTORTION)

[('G63-57307-1111-20704.wav', 'G63-57307-4411-37552.wav'),
 ('G73-53208-1111-20848.wav', 'G73-53208-4411-37696.wav'),
 ('G83-67412-1111-21034.wav', 'G83-67412-4411-37882.wav'),
 ('G61-64600-1111-20658.wav', 'G61-64600-4411-37506.wav'),
 ('G61-67603-1111-20661.wav', 'G61-67603-4411-37509.wav'),
 ('G71-56401-1111-20789.wav', 'G71-56401-4411-37637.wav'),
 ('G81-57212-1111-20930.wav', 'G81-57212-4411-37778.wav'),
 ('G71-58308-1111-20783.wav', 'G71-58308-4411-37631.wav'),
 ('G91-50300-1111-21087.wav', 'G91-50300-4411-37935.wav'),
 ('G81-73609-1111-20979.wav', 'G81-73609-4411-37827.wav'),
 ('G93-65410-1111-21188.wav', 'G93-65410-4411-38036.wav'),
 ('G83-40100-1111-20983.wav', 'G83-40100-4411-37831.wav'),
 ('G91-61406-1111-21106.wav', 'G91-61406-4411-37954.wav'),
 ('G63-63408-1111-20718.wav', 'G63-63408-4411-37566.wav'),
 ('G83-66602-1111-21050.wav', 'G83-66602-4411-37898.wav'),
 ('G91-66602-1111-21128.wav', 'G91-66602-4411-37976.wav'),
 ('G63-71607-1111-20743.wav', 'G63-71607-4411-37591.wav'